---
  title: "Ontologies in Java with Apache Jena"
  description: "Pratical introduction to ontologies with Apache Jean in Java."
  categories: 
    - Java
    - Lecture
    - Ontology
    - Jena
---

# An introduction to OWL Ontologies

Dans le contexte informatique une ontologie est un modèle conceptuel d'un domaine métier. L'objectif de ce modèle est de faciliter le partage d'information à propos de ce domaine en s'appuyant sur des constructions standards. 

Les exemples les plus simples sont les taxonomies, comme par exemple en biologie la classification des êtres vivants. Plus généralement, une ontologie comprend aussi des contraintes à respecter ou des règles permettant d'inférer de nouveaux faits.  

Web Semantique : 
Berners-Lee, T., Hendler, J. & Lassila, O.  (2001).  "The Semantic Web," Scientific American, May 2001. 
Linked Open Data (LOD):

XSD
RDF
RDFS
OWL

https://www.w3.org/TR/2012/REC-owl2-primer-20121211/

Une ontologie décrite en utilisant OWL permet des définir des classes, des propriétés, des individus et des valeurs. 
On distingue :
 * le niveau terminologique (TBox) qui décrit les connaissances générales d'un domaine.
    >"Il existe des départements qui ont des employés et un directeur."
 * Le niveau factuel (ABox) qui décrit une configuration précise 
    >"Pierre est le directeur du département d'informatique. Marie est membre du département d'informatique."

In [1]:
%%loadFromPOM
<dependency>
 <groupId>org.apache.jena</groupId>
 <artifactId>apache-jena-libs</artifactId>
 <version>4.7.0</version>
 <type>pom</type>
</dependency>
<dependency>
 <groupId>ch.qos.logback</groupId>
 <artifactId>logback-classic</artifactId>
 <version>1.4.5</version>
</dependency>
<dependency>
  <groupId>com.github.galigator.openllet</groupId>
  <artifactId>openllet-jena</artifactId>
  <version>2.6.5</version>
</dependency>

EvalException: Exception occurred while running cell magic 'loadFromPOM': Could not find artifact org.apache.jena:apache-jena-libs:jar:4.7.0 in central (https://repo.maven.apache.org/maven2/)

In [2]:
import org.apache.jena.ontology.OntModel;
import org.apache.jena.rdf.model.ModelFactory;
import org.apache.jena.reasoner.ValidityReport;
import org.apache.jena.ontology.OntModelSpec;

import java.io.ByteArrayInputStream;

import org.apache.jena.riot.RDFDataMgr;
import org.apache.jena.riot.Lang;

import ch.qos.logback.classic.util.ContextInitializer;

import openllet.jena.PelletReasonerFactory;

OntModel model;

System.setProperty(ContextInitializer.CONFIG_FILE_PROPERTY, "/home/jovyan/work/javanotebook-full/logback.xml");

IJava.getKernelInstance().getMagics().registerCellMagic("loadOwlModel", (args, body) -> {
    body="""
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix : <http://ebruno.fr/2023/ontologies/test#> .    
"""+body;

    //model = ModelFactory.createOntologyModel(OntModelSpec.OWL_MEM_MINI_RULE_INF);
    if (model == null) 
        model = ModelFactory.createOntologyModel(PelletReasonerFactory.THE_SPEC);
    if (args.contains("reset")) {
        System.err.println("model resetted.");
        model.removeAll();
    }
    model.read(new ByteArrayInputStream(body.getBytes()), null, "TURTLE");
    model.setDerivationLogging(false);
    
    StringBuilder out=new StringBuilder("Model size: " + model.size()+"<BR/>");
    ValidityReport validity = model.validate();
    StringBuilder validityReports = new StringBuilder();
    if (validity.isValid()) {
            out.append("Model is valid<BR/>");
        } else {
            out.append("Conflicts in the model :<BR/><OL>");
            validity.getReports().forEachRemaining(r-> out.append("<LI>"+r+"</LI>"));            
            out.append("</OL>");
        } 
 display(out,"text/markdown");
 return out;
});

CompilationException: 

In [3]:
import org.apache.jena.ontology.OntModel;
import org.apache.jena.rdf.model.ModelFactory;
import org.apache.jena.reasoner.ValidityReport;
import org.apache.jena.ontology.OntModelSpec;
import org.apache.jena.query.*;
import org.apache.jena.rdf.model.Model;

import java.io.ByteArrayInputStream;

import org.apache.jena.riot.RDFDataMgr;
import org.apache.jena.riot.Lang;

import ch.qos.logback.classic.util.ContextInitializer;

IJava.getKernelInstance().getMagics().registerCellMagic("printNS", (args, body) -> {
String sparqlConstructQueryString = String.format("""
PREFIX ns: %s
DESCRIBE ?s {
                ?s  ?o  ?z
        FILTER (isURI(?s) && STRSTARTS(str(?s), str(ns:) ) )
}""",body);                                                                                                            
        Query query = QueryFactory.create(sparqlConstructQueryString) ;
        QueryExecution qexec = QueryExecutionFactory.create(query, model) ;
        Model resultModel = qexec.execDescribe() ;
        qexec.close() ;

        RDFDataMgr.write(System.out, resultModel, Lang.TRIG) ;
    
    String out="";
    
 return out;
});

CompilationException: 

## Overview

### Les classes

In [4]:
%%loadOwlModel
# A individual belonging to a Class
:Mary rdf:type :Person .

# the same individual belonging to another Class
:Mary rdf:type :Woman .

# A class hierarchy
:Woman rdfs:subClassOf :Person .
:Mother rdfs:subClassOf :Woman .

# Two classes can refer to the same set of individuals 
#   and be semantically equivalent.
:Person owl:equivalentClass :Human .
    


EvalException: Undefined cell magic 'loadOwlModel'

### Les propriétés

In [5]:
%%loadOwlModel reset
:hasWife rdfs:domain :Man ;
          rdfs:range  :Woman .

EvalException: Undefined cell magic 'loadOwlModel'

In [6]:
%%printNS
<http://ebruno.fr/2023/ontologies/test#>

EvalException: Undefined cell magic 'printNS'

### Classes énumérées

In [7]:
%%loadOwlModel reset
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:Status rdf:type owl:Class ;
    owl:oneOf (:OnLine :Offline) .

:sensor1 :hasStatus :Online .
        

EvalException: Undefined cell magic 'loadOwlModel'

### Union

In [8]:
%%loadOwlModel reset
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:LegalAgent a owl:Class ;
owl:equivalentClass [
        a owl:Class ;
        owl:unionOf ( :Person :Group )
    ] .

EvalException: Undefined cell magic 'loadOwlModel'

### Intersection

In [9]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:Man a owl:Class ;
owl:equivalentClass [
    a owl:Class ;
    owl:intersectionOf ( :Person :Male )
].

EvalException: Undefined cell magic 'loadOwlModel'

### Négation

In [10]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:Inedible a owl:Class ;
owl:equivalentClass [
  a owl:Class ;
  owl:complementOf :Edible
  ] .
    
:Pierre a :Edible.

EvalException: Undefined cell magic 'loadOwlModel'

### Disjonction

deux classes : Une ressource ne peut pas appartenir aux deux classes en même temps.

In [11]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:Square rdf:type owl:Class ;
        owl:disjointWith :Circle .
    
:c a :Square ; 
   a :Circle . 

EvalException: Undefined cell magic 'loadOwlModel'

In [12]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
                   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
                   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
                   @prefix owl: <http://www.w3.org/2002/07/owl#> .
                   @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
                   :Mary a :Student; a :Professor .
                   :Student a owl:Class .
                   :Professor a owl:Class .
                       
                   :Student rdf:type owl:Class ;
                       owl:disjointWith :Professor .

EvalException: Undefined cell magic 'loadOwlModel'

### Disjonction de plusieurs classes

une ressource ne peut, au plus, appartenir qu’à une seule de ces classes

In [13]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

[] rdf:type owl:AllDisjointClasses ;
   owl:members
   :Square, :Circle, :Triangle .
    
:c a :Square .
:c a :Circle .

EvalException: Undefined cell magic 'loadOwlModel'

In [14]:
%%printNS
<http://ebruno.fr/2023/ontologies/test#>

EvalException: Undefined cell magic 'printNS'

### Union disjointe
diviser une classe en une partition complète de sous classes

In [15]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:Passenger rdf:type owl:Class ;
           owl:disjointUnionOf :Adult, :Child, :Pet .
               
:p1 a :Adult ;             
    a :Child ;
    a :Pet .

EvalException: Undefined cell magic 'loadOwlModel'

In [16]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:John a :Man ;
    :name "John" ;
    :hasSpouse :Mary .
:Mary a :Woman ;
    :name "Mary" ;
    :hasSpouse :John .
:John_jr a :Man ;
    :name "John Jr." ;
    :hasParent :John, :Mary .
:Time_Span a owl:Class .
:event a :Activity ;
    :has_time_span [
        a :Time_Span ;
        :at_some_time_within_date "2018-01-12"^^xsd:date
        ] .
:u129u-klejkajo-2309124u-sajfl a :Person ;
    :name "John Doe" .

EvalException: Undefined cell magic 'loadOwlModel'

## Les propriétés

  1. owl:ObjectProperty sont des relations entre des ressources
  1. owl:DatatypeProperty ont des valeurs littérales (typées)
  1. owl:AnnotationProperty sont ignorées dans les inférences et utilisées pour documenter ou pour des extensions

In [17]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:aSensor :hasName "s1",
    :hasModel,


EvalException: Undefined cell magic 'loadOwlModel'

Une relation peut être symétriques ou non, et/ou réflexive ou non.

AsymmetricProperty

In [18]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:hasSpouse rdf:type owl:ObjectProperty ,
        owl:SymmetricProperty ,
        owl:IrreflexiveProperty .
            
:x :hasSpouse :y.
    
:b :hasSpouse :b.

EvalException: Undefined cell magic 'loadOwlModel'

inverse : deux relations qui existent simultanément en sens inverse (ex. parent_de /
enfant_de)

transitives : une relation qui se propage de proche en proche (ex. Tom ancêtre Jim ancêtre
Jules)

propriétés disjointes : Des relations qui ne peuvent pas exister en même temps sur le même sujet et le même objet

In [19]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:hasAncestor rdf:type owl:ObjectProperty ,
        owl:transitive ;
        owl:inverseOf :hasDescendant .            

:hasSon rdf:type owl:ObjectProperty ;
        owl:disjointwith :hasDaughter.             
            
:Mary :hasHusband :Pierre .
:Mary :hasAncestor :Jean .
:Jean :hasAncestor :Sophie .
            

EvalException: Undefined cell magic 'loadOwlModel'

In [20]:
%%printNS
<http://ebruno.fr/2023/ontologies/test#>

EvalException: Undefined cell magic 'printNS'

In [21]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
          
:hasSon rdf:type owl:ObjectProperty ;
        owl:propertyDisjointWith :hasDaughter.
            
:x :hasSon :y .
:x :hasDaughter :y .

EvalException: Undefined cell magic 'loadOwlModel'

propriétés chaînées
des relations qui mises bout à bout impliquent une autre relation
(ex. parent + frère = oncle)

In [22]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:hasUncle owl:propertyChainAxiom (:hasParent :hasBrother).

:hasGrandparent owl:propertyChainAxiom ( :hasParent :hasParent ).

:x :hasParent :y.
:y :hasParent :z.
:y :hasBrother :y1.

EvalException: Undefined cell magic 'loadOwlModel'

propriétés fonctionnelles
une relation pour laquelle une ressource ne peut avoir qu’une valeur (ex.
naissance)

propriétés inverses fonctionnelles
une relation pour laquelle une même valeur implique la même ressource (ex.
NSS)

In [23]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
@prefix f: <http://example.com/owl/families#> .
@prefix g: <http://example.com/owl/families#> .

f:hasWife rdf:type owl:ObjectProperty ,
        owl:InverseFunctionalProperty ,
        owl:FunctionalProperty ,
        owl:AsymmetricProperty ,
        owl:IrreflexiveProperty ;
    rdfs:domain f:Person ;
    rdfs:range f:Person ;
    rdfs:subPropertyOf f:loves ;
    rdfs:range f:Woman ;
    rdfs:subPropertyOf f:hasSpouse ;
    rdfs:domain f:Man .
        
:x f:hasWife :y .

EvalException: Undefined cell magic 'loadOwlModel'

identification par des clés
deux instances qui ont même(s) valeur(s) de clé(s) sont en réalité la même
instance

In [24]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:Person owl:hasKey ( :hasSSN ) .
    
:x :hasSSN "1234" . 
:y :hasSSN "1234" .
:x :hasName "Pierre" .

EvalException: Undefined cell magic 'loadOwlModel'

In [25]:
%%printNS
<http://ebruno.fr/2023/ontologies/test#>

EvalException: Undefined cell magic 'printNS'

## Negative properties assertion

In [26]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

[]  rdf:type               owl:NegativePropertyAssertion ;
     owl:sourceIndividual   :Bill ;
     owl:assertionProperty  :hasWife ;
     owl:targetIndividual   :Mary .
        

EvalException: Undefined cell magic 'loadOwlModel'